Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [1]:
!wget 'https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_' -O data.zip

--2022-06-07 20:01:57--  https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_
Resolving drive.google.com (drive.google.com)... 74.125.142.101, 74.125.142.138, 74.125.142.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ksm281rums9e7d2ivsjst9b8vlpp3nik/1654632075000/14904333240138417226/*/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_?e=download [following]
--2022-06-07 20:01:58--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ksm281rums9e7d2ivsjst9b8vlpp3nik/1654632075000/14904333240138417226/*/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_?e=download
Resolving doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleusercontent.com)... 142.250.107.132, 2607:f8b0:400e:c0d::84
Connecting to doc-04-c0-docs.googleusercontent.com (doc-04-c0-doc

In [2]:
!unzip data.zip
!rm data.zip

Archive:  data.zip
  inflating: marketing description.txt  
  inflating: marketing_campaign.csv  
  inflating: subscribers.csv         
  inflating: users.csv               


In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from statsmodels.stats import proportion

In [5]:
data = pd.read_csv('marketing_campaign.csv')
data.head()

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True


In [7]:
data.info()  # missing values in converted. As far as we don't know the truth impact on user behaviour we set False for these nans

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   date_served         10021 non-null  object
 2   marketing_channel   10022 non-null  object
 3   variant             10037 non-null  object
 4   language_displayed  10037 non-null  object
 5   converted           10022 non-null  object
dtypes: object(6)
memory usage: 470.6+ KB


In [26]:
data.converted.value_counts()

False    8961
True     1076
Name: converted, dtype: int64

In [29]:
data.variant.value_counts() # almost similar sized groups

control            5091
personalization    4946
Name: variant, dtype: int64

In [8]:
data['converted'].fillna(False, inplace=True)

In [30]:
alpha = 0.05  # base significance level
k1 = data[data['variant']=='control']['converted'].sum()
n1 = data[data['variant']=='control'].shape[0]
k2 = data[data['variant']=='personalization']['converted'].sum()
n2 = data[data['variant']=='personalization'].shape[0]
print(f'Size of personalization group / conversion: {k2}/{n2}={k2/n2:.4f}, size of control group/conversion: {k1}/{n1}={k1/n1:.4f}.')
print(f'')

Size of personalization group / conversion: 705/4946=0.1425, size of control group/conversion: 371/5091=0.0729.



**В тестовой группе (personalization) конверсия (converted) выше чем в контрольной группе (control), необходимо установить является ли эта разница статистически значимой**

Сформулируем основную и альтернативную гипотезы:
- "нулевая" $H_0$ - наблюдаемые различия не имеют статистически значимых различий
- "альтернативная" $H_1$ - различия значимы\
Выберем для анализа параметрический (предполагающий нормальное распределение в данных) и непараметрический тесты: $Z-test$ (Test for proportions based on normal (z) test) и $\chi{^2}$
- установим стандартный уровень значимости $\alpha = 0.05$
- выясним значение статистики и $p_{value}$
- сделаем вывод о возможности принятия $H_0$ или отказа от нее в пользу $H_1$

#### Z-test (proportions, In the one and two sample cases with two-sided alternative, this test produces the same p-value as proportions_chisquare, since the chisquare is the distribution of the square of a standard normal distribution.)

In [31]:
z_score, pvalue = proportion.proportions_ztest(np.array([k1, k2]), np.array([n1, n2]))
if pvalue <= alpha:
    print("Наблюдаемые различия значимы (альтернативная гипотеза)")
else:
    print("Наблюдаемые значения не имеют статистически значимых различий")
print(f'Z_score={z_score:.3f} p_value={pvalue:.3f}')

Наблюдаемые различия значимы (альтернативная гипотеза)
Z_score=-11.279 p_value=0.000


#### Chi squared test

In [32]:
chisquared, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), np.array([n1, n2]))
if pvalue <= alpha:
    print("Наблюдаемые различия значимы (альтернативная гипотеза)")
else:
    print("Наблюдаемые значения не имеют статистически значимых различий")
print(f'Z_score={chisquared:.3f} p_value={pvalue:.3f}')
pd.DataFrame(data=table[0])

Наблюдаемые различия значимы (альтернативная гипотеза)
Z_score=127.213 p_value=0.000


,0,1
0,371,4720
1,705,4241


#### scipy library

In [19]:
from scipy import stats

In [20]:
person = data.loc[data['variant'] == 'personalization', 'converted'].values.astype(int)
control = data.loc[data['variant'] == 'control', 'converted'].values.astype(int)

In [21]:
person.mean(), control.mean(), len(person), len(control)

(0.14253942579862516, 0.07287369868395208, 4946, 5091)

#### Ради интереса попробуем T-test для вещественных выборок (в курсе по мат. статистике использовался именно он для A/B тестирования, что еще тогда мне показалось странным)

In [22]:
stats.ttest_ind(control, person, equal_var=False)  # Student test, that usually performed to compare means of two normally distributed samples of real values, results are the same

Ttest_indResult(statistic=-11.302765842140486, pvalue=1.9939161483253462e-29)

In [23]:
table = data.pivot_table(values='user_id', index='variant', columns='converted', aggfunc='count')
table

converted,False,True
variant,,
control,4720,371
personalization,4241,705


$\chi^2$

In [24]:
stats.chi2_contingency(table, correction=False)[:2]

(127.21277698468771, 1.6688675207808698e-29)